In [1]:
import os
import glob
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Ensure plots are displayed inline in Jupyter
%matplotlib inline

In [ ]:
# --- 1. SET YOUR DIRECTORY PATHS HERE ---
# Replace these with the actual paths to your data
dir_method1 = '/ess/scratch/scratch1/rachelgordon/zp-test/fastMRI_breast_001_2/'
dir_method2 = '/ess/scratch/scratch1/rachelgordon/interpolation-test/fastMRI_breast_001_2/'
# ---

# --- 2. Find and sort the NIfTI files ---
# The pattern 'slice_*_frame_*.nii' should match your file naming convention.
# Use glob to find all files matching the pattern and sort them.
# Sorting is crucial to ensure slice_000, slice_001, etc., are in order.
search_pattern = 'slice_*_frame_*.nii'
files_method1 = sorted(glob.glob(os.path.join(dir_method1, search_pattern)))
files_method2 = sorted(glob.glob(os.path.join(dir_method2, search_pattern)))

# --- 3. Verification Step ---
# Check if both directories contain the same number of slices
if not files_method1 or not files_method2:
    print("Warning: One or both directories are empty or the path is incorrect.")
    print(f"Files found in '{dir_method1}': {len(files_method1)}")
    print(f"Files found in '{dir_method2}': {len(files_method2)}")
elif len(files_method1) != len(files_method2):
    print("Warning: The directories contain a different number of slices.")
    print(f"Slices for Method 1: {len(files_method1)}")
    print(f"Slices for Method 2: {len(files_method2)}")

# Determine the number of slices from the file list
num_slices = len(files_method1)


# --- 4. Define the plotting function ---
def compare_slices(slice_index):
    # Make sure we don't go out of bounds if lists are empty
    if slice_index >= num_slices:
        print("Slice index is out of range.")
        return

    # Get the full path for the selected slice for both methods
    filepath1 = files_method1[slice_index]
    filepath2 = files_method2[slice_index]
    
    # Load the NIfTI images
    try:
        nifti_img1 = nib.load(filepath1)
        nifti_img2 = nib.load(filepath2)
    except FileNotFoundError as e:
        print(f"Error loading file: {e}")
        return

    # Get the image data as a NumPy array
    # .get_fdata() is commonly used; it returns data in a standard orientation
    slice_data1 = nifti_img1.get_fdata()
    slice_data2 = nifti_img2.get_fdata()
    
    # Create the plot
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # Display the slice from Method 1
    axes[0].imshow(slice_data1, cmap='gray')
    axes[0].set_title(f'Zero-Padding\n{os.path.basename(filepath1)}')
    axes[0].axis('off')

    # Display the slice from Method 2
    axes[1].imshow(slice_data2, cmap='gray')
    axes[1].set_title(f'Interpolation\n{os.path.basename(filepath2)}')
    axes[1].axis('off')
    
    plt.show()

# --- 5. Create the interactive slider ---
# The slider will only be created if slices were found
if num_slices > 0:
    interact(
        compare_slices, 
        slice_index=widgets.IntSlider(
            min=0, 
            max=num_slices - 1, 
            step=1, 
            value=num_slices // 2, # Start in the middle
            description='Slice Index',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='50%')
        )
    );
else:
    print("\nCould not create interactive viewer because no slices were found.")

interactive(children=(IntSlider(value=96, description='Slice Index', layout=Layout(width='50%'), max=191, styl…